In [2]:
# Use centrailzed training to compare with federated learning
epochs = 30
n_train_items = 12800

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import numpy as np
import opacus
from opacus import PrivacyEngine
import copy
# import torchcsprng as csprng


In [4]:
class Arguments():
    def __init__(self):
        self.batch_size = 250
        self.test_batch_size = 64
        self.epochs = epochs
        self.lr = 0.25
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# random_number_generator = csprng.create_random_device_generator(
#                 "/dev/urandom")

In [5]:
train_dataloader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)

    
#---

# train_dataloader = [
#         ((data), (target))
#         for i, (data, target) in enumerate(train_loader)
#         if i < n_train_items / args.batch_size
#     ]


# test_dataloader = [
#         ((data), (target))
#         for i, (data, target) in enumerate(test_loader)
#         if i < n_train_items / args.batch_size
#     ]


In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 8, 2, padding=3)
        self.conv2 = nn.Conv2d(16, 32, 4, 2)
        self.fc1 = nn.Linear(32 * 4 * 4, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        # x of shape [B, 1, 28, 28]
        x = F.relu(self.conv1(x))  # -> [B, 16, 14, 14]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 16, 13, 13]
        x = F.relu(self.conv2(x))  # -> [B, 32, 5, 5]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 32, 4, 4]
        x = x.view(-1, 32 * 4 * 4)  # -> [B, 512]
        x = F.relu(self.fc1(x))  # -> [B, 32]
        x = self.fc2(x)  # -> [B, 10]
        return x

    def name(self):
        return "SampleConvNet"

In [24]:
def train(args, device, model, train_dataloader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_dataloader):
        data,target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        o_model= copy.deepcopy(model)
        optimizer_nodp = optim.SGD(o_model.parameters(), lr=args.lr)
        output = o_model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer_nodp.step()
        
        oms = o_model.state_dict()
        ms = model.state_dict()
        for key in ms:
            oms[key] += (ms[key]-oms[key])*0.1
        model.load_state_dict(oms)

        
        
        
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_dataloader.dataset),
                100. * batch_idx / len(train_dataloader), loss.item()))

In [22]:
def test(args, device, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():

            
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / (len(test_loader.dataset))))


In [27]:
#%%time
model = Net().to(device)
optimizer =  optim.SGD(model.parameters(), lr=args.lr)

privacy_engine = PrivacyEngine(
                model,
                batch_size=args.batch_size,
                sample_size=len(train_dataloader.dataset),
                alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64)),
                noise_multiplier=6,
                max_grad_norm=1.5,)
privacy_engine.attach(optimizer)

for epoch in range(1, args.epochs + 1):
    train(args, device, model, train_dataloader, optimizer, epoch)
    test(args, device, model, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

/opt/conda/envs/DP/lib/python3.7/site-packages/opacus/privacy_engine.py:111: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.321145
Train Epoch: 1 [7500/60000 (12%)]	Loss: 1.594238
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.607587
Train Epoch: 1 [22500/60000 (38%)]	Loss: 0.414686
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.240218
Train Epoch: 1 [37500/60000 (62%)]	Loss: 0.328623
Train Epoch: 1 [45000/60000 (75%)]	Loss: 0.205420
Train Epoch: 1 [52500/60000 (88%)]	Loss: 0.215676

Test set: Average loss: 2.5372, Accuracy: 4327/10000 (43%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.889981
Train Epoch: 2 [7500/60000 (12%)]	Loss: 0.153768
Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.113291
Train Epoch: 2 [22500/60000 (38%)]	Loss: 0.250202
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.166438
Train Epoch: 2 [37500/60000 (62%)]	Loss: 0.254458
Train Epoch: 2 [45000/60000 (75%)]	Loss: 0.196098
Train Epoch: 2 [52500/60000 (88%)]	Loss: 0.167001

Test set: Average loss: 0.1880, Accuracy: 9468/10000 (95%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.169582
Train Epoch: 3 [7500/60000 (12%)]	Loss: 0.103

KeyboardInterrupt: 